In [ ]:
import json
import sys
import pandas as pd
import numpy as np
import plotly.express as px
# import pandas as pd
from datetime import datetime
from tronscan.client import TronscanClient

sys.path.append('..')

from env import settings

%load_ext autoreload
%autoreload 2

In [ ]:
client = TronscanClient(api_key=settings.TRONSCAN_API_TOKEN)
my_addr = settings.TRON_ADDRS[0]
my_addr_2 = settings.TRON_ADDRS[1]

In [ ]:
transfers = list(client.all_transfers(my_addr)) + list(client.all_transfers(my_addr_2))

In [ ]:
len(transfers)

In [ ]:
transfers[0]

In [ ]:
usdt_transfers = [
    ((1 if t['to_address'] in (my_addr, my_addr_2) else -1) * float(int(t['quant']) / 10**t['tokenInfo']['tokenDecimal']), datetime.fromtimestamp(int(t['block_ts']) / 1000))
    for t in transfers if t['tokenInfo']['tokenId'] == 'TR7NHqjeKQxGTCi8q8ZY4pL8otSzgjLj6t'
]

In [ ]:
len(usdt_transfers)

In [ ]:
df = pd.DataFrame(usdt_transfers, columns=["amount", "date"])

In [ ]:
df['amount'].sum()

In [ ]:
fig = px.bar(df, x="date", y="amount", title="USDT transfers")
fig.update_layout(showlegend=False)
fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='black')
fig.update_traces(marker_color=df['amount'].apply(lambda x: 'green' if x > 0 else 'red'))
fig.update_traces(width=500000000)
fig.show()

In [ ]:
def show_incomes_expenses(df, grouping_freq="M"):
    incomes = df[df['amount'] > 0]
    expenses = df[df['amount'] < 0]

    incomes_grouped = incomes.groupby(pd.Grouper(key="date", freq=grouping_freq)).sum().reset_index()
    expenses_grouped = expenses.groupby(pd.Grouper(key="date", freq=grouping_freq)).sum().reset_index()
    together = pd.concat([incomes_grouped, expenses_grouped], axis=0).sort_values("date")

    fig = px.bar(together, x="date", y="amount", title="USDT transfers")
    fig.update_layout(showlegend=False)
    fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='black')
    fig.update_traces(marker_color=together['amount'].apply(lambda x: 'green' if x > 0 else 'red'))
    fig.update_traces(width=500000000)
    fig.show()
    
    


In [ ]:
show_incomes_expenses(df, "M")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime, timedelta

start = datetime(2024, 3, 6, 9, 0, 3, 451571)

payments = [
    (datetime(2024, 4, 9, 9, 0, 0, 0), 43.5),
]

daily_range = np.arange(start, start + timedelta(days=180), timedelta(days=1)).astype(datetime)
timestamps = np.vectorize(lambda x: x.timestamp())(daily_range)
timestamps -= timestamps.min()


def debt_with_interest(X, r, t, seconds_per_t=60*60*24*30):
    return X * np.exp(r * t / seconds_per_t)

def linear_debt(X, r, t, seconds_per_t=60*60*24*30):
    return X * (1 + r * t / seconds_per_t)


fig = plt.figure(figsize=(15, 5))

dr = daily_range
ts = np.vectorize(lambda x: x.timestamp())(dr)
ts -= ts.min()

plt.plot(dr, debt_with_interest(500, linear_debt(0.08, 0.5, ts), ts),
         label='Interest rate starts at 7% and increases by 5% each month, continuous compounding')
plt.plot(dr, debt_with_interest(500, 0.15, ts), label='15% interest per month, continuous compounding')
plt.plot(dr, linear_debt(500, 0.20, ts), label='20% interest per month, no compounding')
plt.ylabel('Debt, $')
plt.xlabel('Date')
plt.grid()
plt.legend()
debt_with_interest(500, linear_debt(0.08, 0.5, ts), ts)
from datetime import datetime, timedelta

now = datetime(2024, 3, 6, 9, 0, 3, 451571)
daily_range = np.arange(now, now + timedelta(days=365), timedelta(days=1)).astype(datetime)

def debt_with_interest(X, r, t, seconds_per_t=60*60*24*30):
    return X * np.exp(r * t / seconds_per_t)

def linear_debt(X, r, t, seconds_per_t=60*60*24*30):
    return X * (1 + r * t / seconds_per_t)

timestamps = np.vectorize(lambda x: x.timestamp())(daily_range)
timestamps -= timestamps.min()


def current_debt_of_leonid():
    start = datetime(2024, 3, 6, 9, 0, 3, 451571)
    cur_debt = 500
    cur_interest_rate = 0.08

    for t, payment in payments:
        dt = (t - start).total_seconds()
        cur_interest_rate = linear_debt(cur_interest_rate, 0.5, dt)
        cur_debt = debt_with_interest(cur_debt, cur_interest_rate, dt)
        cur_debt -= payment
        start = t

    t = (datetime.now() - start).total_seconds()
    return debt_with_interest(cur_debt, linear_debt(cur_interest_rate, 0.5, t), t)


current_debt_of_leonid()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def pb_posterior(pb_prior, pt):
    return pb_prior * pt / (pb_prior * pt + (1 - pb_prior) * (1 - pt))


In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

# Make data.
pb_prior = np.linspace(0, 1, 100)
pt = np.linspace(0, 1, 100)
pb_prior, pt = np.meshgrid(pb_prior, pt)
pb_post = pb_posterior(pb_prior, pt)

# Plot the surface.
surf = ax.plot_surface(pb_prior, pt, pb_post, cmap='coolwarm')

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

In [ ]:
plt.imshow(pb_post, cmap='coolwarm', interpolation='nearest', origin='lower')
plt.xlabel('Prior belief')
plt.ylabel('Probability of Alice lying')
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(pb_post >= pb_prior, cmap='coolwarm', interpolation='nearest', origin='lower')
plt.xlabel('Prior belief')
plt.ylabel('Probability of Alice lying')
plt.colorbar()
plt.show()